In [1]:
import numpy as np
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from mpl_toolkits import mplot3d
import seaborn as sns
from sklearn.decomposition import PCA

ModuleNotFoundError: No module named 'seaborn'

Going to start looking at ride_id to see correlations between ride distance and duration and prime time

In [ ]:
ride_id = pd.read_csv("ride_ids.csv")

In [ ]:
ride_id

driver id, ride id, ride_distance, ride_duration, ride_prime_time, total money, response time 

Base Fare
Cost per Mile Cost per Minute Service Fee Minimum Fare Maximum Fare
Submission instructions:
$2.00 $1.15 $0.22 $1.75

In [ ]:
ride_id["ride_cost"] = ride_id['ride_distance'] * 1.15 * 0.000621371 + ride_id["ride_duration"] * 0.22 / 60 + 3.75
ride_id[ride_id["ride_cost"] < 5] = 5
ride_id

In [ ]:
driver_id = pd.read_csv("driver_ids.csv")

In [ ]:
driver_id

Make new database of driver along with the ride_ids as

Distance x Duration

In [ ]:
plt.scatter(ride_id["ride_distance"], ride_id["ride_duration"], alpha=0.5)

In [ ]:
ride_id["ride_prime_time"]

Duration x Prime Time 

In [ ]:
plt.scatter(ride_id.iloc[:,3], ride_id.iloc[:,4], alpha=0.5)

Migh tmake a driver/ride histogram to see the avg number of rides a driver does, should be normally distributed

In [ ]:
ride_timestamps= pd.read_csv("ride_timestamps.csv")

In [ ]:
ride_timestamps2 = ride_timestamps
a = ride_timestamps2[ride_timestamps2.index % 5== 0].reset_index()
b = ride_timestamps2[ride_timestamps2.index % 5 == 1].reset_index()
c = ride_timestamps2[ride_timestamps2.index % 5== 2].reset_index()
d = ride_timestamps2[ride_timestamps2.index % 5 == 3].reset_index()
e = ride_timestamps2[ride_timestamps2.index % 5== 4].reset_index()
ride_timestamps2 = ride_timestamps2[ride_timestamps2.index % 5 == 0]
s = pd.DataFrame(ride_timestamps2["ride_id"]).reset_index()
s["requested_at"] = a["timestamp"]
s["accepted_at"] = b["timestamp"]
s["arrived_at"] = c["timestamp"]
s["picked_up_at"] = d["timestamp"]
s["dropped_off_at"] = e["timestamp"]



In [ ]:
s["time_waited"] = s["picked_up_at"].map(lambda a : datetime.strptime(a, "%Y-%m-%d %H:%M:%S")) - s["requested_at"].map(lambda a : datetime.strptime(a, "%Y-%m-%d %H:%M:%S"))
s["response_time"] =  s["accepted_at"].map(lambda a : datetime.strptime(a, "%Y-%m-%d %H:%M:%S")) - s["requested_at"].map(lambda a : datetime.strptime(a, "%Y-%m-%d %H:%M:%S"))

In [ ]:
s["time_waited"] = s["time_waited"].apply(lambda a : a.seconds)
s["response_time"] = s["response_time"].apply(lambda a : a.seconds)

In [ ]:
#df_merge_col = pd.merge(df_row, df3, on='id')
merge = pd.merge(ride_id, s[["ride_id", "time_waited", "response_time"]], on ="ride_id")

In [ ]:
merge

In [ ]:
kmeans_data = merge[["ride_distance", "ride_duration", "ride_prime_time", "ride_cost", "time_waited", "response_time"]]

kmeans_data_no_out = kmeans_data[kmeans_data.ride_cost < 600]

In [ ]:
kmeans_data_no_out = kmeans_data[kmeans_data.ride_cost < 600]

In [ ]:
km = KMeans(
    n_clusters=3, init='random',
    n_init=10, max_iter=300, 
    tol=1e-04, random_state=0
)
y_km = km.fit_predict(kmeans_data)

In [ ]:
km.cluster_centers_

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

zs = kmeans_data_no_out['ride_distance']
xs = kmeans_data_no_out['ride_duration']
ys = kmeans_data_no_out['ride_cost']
ax.scatter(xs, ys, zs, s=50, alpha=0.6, edgecolors='w')

ax.set_xlabel('Ride Distance')
ax.set_ylabel('Ride Duration')
ax.set_zlabel('Ride Cost')


In [ ]:
cost_mean = kmeans_data_no_out['ride_cost'].median()
kmeans_data_no_out.loc[kmeans_data_no_out.ride_cost < cost_mean, 'ride_cost']  = 0
kmeans_data_no_out.loc[kmeans_data_no_out.ride_cost >= cost_mean, 'ride_cost']  = 1

In [ ]:
kmeans_data_no_out

In [ ]:
jp = sns.pairplot(kmeans_data_no_out, x_vars=["ride_distance"], y_vars=["ride_duration"], size=4.5,
                  hue="ride_cost", palette={0: "#FF9999", 1: "#FFE888"},
                  plot_kws=dict(edgecolor="k", linewidth=0.5))

In [ ]:
kmeans_corr = kmeans_data.corr()

In [ ]:
kmeans_corr

In [ ]:
import numpy as np 
from pandas import DataFrame
import matplotlib.pyplot as plt

plt.pcolor(kmeans_corr)
plt.yticks(np.arange(0.5, len(kmeans_corr.index), 1), kmeans_corr.index)
plt.xticks(np.arange(0.5, len(kmeans_corr.columns), 1), kmeans_corr.columns)
plt.show()

In [ ]:
# intialize pca and logistic regression model
pca = PCA(n_components=3)
pca_data = pca.fit_transform(kmeans_data)

In [ ]:
pca_data

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

xs = pca
ys = kmeans_data_no_out['ride_duration']
zs = kmeans_data_no_out['ride_cost']
ax.scatter(xs, ys, zs, s=50, alpha=0.6, edgecolors='w')

ax.set_xlabel('Ride Distance')
ax.set_ylabel('Ride Duration')
ax.set_zlabel('Ride Cost')